In [1]:
import multiprocessing
from multiprocessing.shared_memory import SharedMemory
import numpy as np
from fastplotlib import Plot
import tifffile

In [2]:
# file path
video_path = "/data/kushal/cortex-learning/2p-trial-exps/eaf3/2022-12-18/calcium/s4/concat_s4.tif"

In [3]:
from mesmerize_core import load_batch

In [4]:
batch_path = "/data/kushal/cortex-learning/2p-trial-exps/eaf3/mesmerize-batch/batch.pickle"
df = load_batch(batch_path)
df.iloc[9]["params"]

{'main': {'fr': 15,
  'p': 1,
  'nb': 2,
  'merge_thr': 0.85,
  'rf': 24,
  'stride': 12,
  'K': 10,
  'gSig': (6, 6),
  'ssub': 1,
  'tsub': 1,
  'method_init': 'greedy_roi',
  'min_SNR': 1.0,
  'rval_thr': 0.8,
  'use_cnn': True,
  'min_cnn_thr': 0.8,
  'cnn_lowest': 0.1,
  'decay_time': 1.0},
 'refit': True}

In [5]:
df.iloc[3]["params"]

{'main': {'max_shifts': (96, 96),
  'strides': (96, 96),
  'overlaps': (48, 48),
  'max_deviation_rigid': 3,
  'border_nan': 'copy',
  'pw_rigid': True,
  'gSig_filt': None,
  'niter_rig': 4}}

In [6]:
# reading video into RAM
calcium = tifffile.imread(video_path)

In [7]:
calcium.shape

(8410, 512, 512)

In [8]:
calcium.nbytes

4409262080

### create a shared memory and dump frames in

In [9]:
def create_shared_memory(shm_name, data):
    try:
        shm = SharedMemory(name=shm_name, create=True, size=data.nbytes)
    except:
        shm = SharedMemory(name=shm_name)
        shm.close()
        shm.unlink()
        shm = SharedMemory(name=shm_name, create=True, size=data.nbytes)
    return shm

In [10]:
# goal: put frames into shared memory
# grab those frames from another kernel and put it through onacid 

In [11]:
shm = create_shared_memory("shared-mem", calcium)

In [12]:
buffer_data = np.ndarray(calcium.shape, dtype=calcium.dtype, buffer=shm.buf)
buffer_data[:] = calcium